In [17]:
import pandas as pd
import numpy as np
import rdkit

In [18]:
df = pd.read_csv('./data/aaa.tsv', sep='\t')

all_smiles = df['Smiles'].unique()

In [19]:
data = pd.read_csv('./logK.csv')
data.head()

,id,smiles,lgK
0,0,CC(C)(O)C(=O)O,3.41
1,1,CCC(O)(CC)C(=O)O,3.25
2,2,O=C(O)C1(O)CCCC1,3.22
3,3,O=C(O)C1(O)CCCCC1,2.78
4,4,O=C(O)C(O)c1ccccc1,2.82


In [20]:
from collections import Counter
from rdkit.Contrib.SA_Score import sascorer
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from rdkit.Chem.Descriptors import ExactMolWt

def check_molecule(smiles):
    # Разрешенные элементы и элементы для подсчета атомов
    allowed_elements = {'C', 'H', 'O', 'N', 'P', 'S'}
    count_elements = {'O', 'N', 'P', 'S'}

    # Счетчик элементов в молекуле
    elements_counter = Counter()
    temp_element = '' # Временная переменная для хранения элементов более одного символа

    for char in smiles:
        if char.isalpha():
            # Проверяем, является ли элемент двухбуквенным (например, Cl) - не применимо к данным элементам, но полезно для общности
            if temp_element:
                # Предыдущий элемент был начат, завершаем его
                elements_counter[temp_element] += 1
                temp_element = ''
            if char.isupper():
                # Начало нового элемента
                elements_counter[char] += 1
            else:
                # Продолжение текущего элемента
                temp_element += char
        elif temp_element:
            # Если был накоплен временный элемент, добавляем его в счетчик
            elements_counter[temp_element] += 1
            temp_element = '' # Сброс временного элемента

    # Проверяем, входит ли последний элемент в счетчик, если цикл завершился на букве
    if temp_element:
        elements_counter[temp_element] += 1

    # Проверка условий
    # 1. Все элементы в молекуле должны быть из разрешенного списка
    if not all(element in allowed_elements for element in elements_counter):
        return False

    # 2. Молекула должна включать не менее трех разных элементов из списка
    if sum(element in allowed_elements for element in elements_counter) < 3:
        return False

    # 3. Суммарно не более 12 атомов элементов O, N, P, S
    if sum(count for element, count in elements_counter.items() if element in count_elements) > 12:
        return False
    
    if smiles in data["smiles"].tolist():
      return False

    weight = ExactMolWt(Chem.MolFromSmiles(smiles))
    if weight > 500: return False

    if sascorer.calculateScore(Chem.MolFromSmiles(smiles))>5: return False


    return True

In [21]:
from rdkit import Chem
from tqdm.auto import tqdm

p = []

for s in tqdm(all_smiles):
    try: 
        Chem.MolFromSmiles(s)
        if check_molecule(s):
            p.append(s)
    except:
        pass
        
all_smiles = p

100%|██████████| 658524/658524 [02:03<00:00, 5327.66it/s]


In [22]:
len(p)

9639

In [23]:
test_df = pd.DataFrame({'smiles': all_smiles})

In [24]:
test_df.head()

,smiles
0,CCCCCCCCCCCCCCCCCCSCC(O)C1OC(O)=C(OC)C1=O
1,O=C(O)C1CCN(C(=O)C[C@H]2CC[C@]3(CC2)OO[C@]2(O3...
2,CCOP(=O)(OCC)[C@]1(C#N)SC[C@@H](CN(C)C)CS1.O=C...
3,CCCCCCCCCCCCCCCC(=O)OCCCOP(=O)(O)OC[C@H](N)C(=O)O
4,CC(=O)O[C@H]1CC[C@@]2(C)[C@@H](CC[C@@H]3[C@@H]...


In [25]:
from pred import CatLgKPredictor

In [26]:
model = CatLgKPredictor('./mymodel')

In [27]:
preds = model.predict_lgK(test_df)

In [28]:
new_df = pd.DataFrame({'smiles': all_smiles, 'lgK': preds})

In [29]:
new_df.lgK.max()

21.843657974936278

In [30]:
new_df.sort_values(by='lgK', ascending=False)

,smiles,lgK
374,CC(O)CN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(CC(=O)O)CC1,21.843658
243,O=C(O)CN1CCN(CCN(CC(=O)O)CC(=O)O)CCN(CC(=O)O)CC1,18.389990
6130,CCOP(=O)(O)CN1CCNCCN(CP(=O)(O)OCC)CCNCC1,17.118992
8806,O=C(O)CN(CC(=O)O)C1CCCCC1N(CC(=O)O)CC(=O)NCCN=C=S,15.553966
8277,NCCCN1CCN(CC(=O)O)CCN(CCCN)CCN(CC(=O)O)CC1,15.499535
...,...,...
7975,CC(C)=CCC/C(C)=C/CC/C(C)=C/CC/C=C(\C)CC/C=C(\C...,5.611275
7683,CCCCCCCCCCCCCCCCCCOCC(O)COCCC[N+](C)(C)C,5.575373
8091,CCCCCCCCCCCCCCCCCCOCC(O)COCCC[N+](C)(C)C.[OH-],5.552443
3290,CCCCCCCCCCCCCCC1COC(COCCCCCC[N+](C)(C)C)C1,5.330629


In [31]:
new_df.to_csv('parsed400_500.csv', index=False)